In [9]:
#| default_exp routes.slack

# Slack Routes
> route functions for interacting with Slack via slackbolt

## To create a slackbot
1. Go to your Slack App settings (api.slack.com/apps)
2. Under "Event Subscriptions", enable events
3. Subscribe to these bot events:
   - message.channels
   - app_mentions

4. Also verify that your bot has been:
   - Added to the channel you're testing in
   - Given the necessary OAuth scopes:
      - chat:write
      - app_mentions:read
      - channels:history

5. Deploy bot to a channel and 
6. update .env with requisite SLACK_BOT_TOKEN, SLACK_SIGNING_SECRET, SLACK_APP_TOKEN

In [19]:
#| export
from slack_bolt.async_app import AsyncApp as AsyncSlackApp
from slack_sdk.errors import SlackApiError

from agent_mafia.client.ResponseGetData import ResponseGetDataSlack

In [21]:
#| exporti
from agent_mafia.client.MafiaError import MafiaError 
import os
from nbdev.showdoc import patch_to

In [31]:
#| exporti

@patch_to(AsyncSlackApp)
async def who_am_i(self):
    res = await self.client.auth_test()

    return res

    return ResponseGetDataSlack(
        is_success=res["ok"],
        response=res['data'],
        status = res['status'],
        app = self
    )


In [37]:
# | export

default_slack_app = AsyncSlackApp(
    token=os.environ["SLACK_BOT_TOKEN"],
    signing_secret=os.environ["SLACK_SIGNING_SECRET"],
)

res = await default_slack_app.who_am_i()
type(res)
# channel_id = "C08JLCQQYAU"

slack_sdk.web.async_slack_response.AsyncSlackResponse

In [14]:
#| export

async def get_channel_history(async_app: AsyncSlackApp, channel_id: str):
    # Store conversation history
    # ID of the channel you want to send the message to

    try:
        # Call the conversations.history method using the WebClient
        # conversations.history returns the first 100 messages by default
        # These results are paginated, see: https://api.slack.com/methods/conversations.history$pagination
        res = await async_app.client.conversations_history(channel=channel_id)

        return res 


    except SlackApiError as e:
        raise MafiaError(exception = e, message = "Error fetching conversation history")


In [15]:
print(await get_channel_history(default_slack_app, channel_id=channel_id))

{'ok': True, 'messages': [{'user': 'U012A577URL', 'type': 'message', 'ts': '1742232642.381019', 'client_msg_id': '10541cba-2e3a-4bcd-973c-15cd7d6e4aee', 'text': '<@U08J0GT9QAX> what is partitioning?', 'team': 'T012H3JSC83', 'thread_ts': '1742232642.381019', 'reply_count': 2, 'reply_users_count': 1, 'latest_reply': '1742232661.572449', 'reply_users': ['U08J0GT9QAX'], 'is_locked': False, 'subscribed': False, 'blocks': [{'type': 'rich_text', 'block_id': '6dfEV', 'elements': [{'type': 'rich_text_section', 'elements': [{'type': 'user', 'user_id': 'U08J0GT9QAX'}, {'type': 'text', 'text': ' what is partitioning?'}]}]}]}, {'subtype': 'channel_join', 'user': 'U03EF0517ED', 'text': '<@U03EF0517ED> has joined the channel', 'type': 'message', 'ts': '1742232507.941979'}, {'user': 'U012A577URL', 'type': 'message', 'ts': '1742232427.416539', 'client_msg_id': '189b8f2f-6c68-47b4-bd3b-391c4030e93c', 'text': '<@U08J0GT9QAX> - what is pdp', 'team': 'T012H3JSC83', 'thread_ts': '1742232427.416539', 'reply_

In [16]:
# | export
async def update_slack_message(
    async_app: AsyncSlackApp,
    channel_id: str,
    message_id: str,  # message timestamp,
    new_text,  # str,
    debug_prn: bool = False
):
    """
    Updates an existing Slack message.

    Args:
        channel_id (str): The ID of the channel where the message is located.
        timestamp (str): The timestamp of the message to update.
        new_text (str): The new text content for the message.
    """
    try:
        return await async_app.client.chat_update(channel=channel_id, ts=message_id, text=new_text)
        if debug_prn:
            print(f"Message updated successfully in channel {channel_id} at {message_id}")
    except SlackApiError as e:
        raise MafiaError(exception = e) from e

In [17]:

await update_slack_message(default_slack_app, "C08JLCQQYAU", "1627480000.000100", "Hello, World!")

MafiaError: 💀  The request to the Slack API failed. (url: https://slack.com/api/chat.update, status: 200)
The server responded with: {'ok': False, 'error': 'message_not_found'}
An exception of type SlackApiError occurred. Arguments: The request to the Slack API failed. (url: https://slack.com/api/chat.update, status: 200)
The server responded with: {'ok': False, 'error': 'message_not_found'}

In [ ]:
#| export
def post_initial_message(channel_id, initial_text, app):
    try:
        result = app.client.chat_postMessage(channel=channel_id, text=initial_text)
        return result["ts"]  # return timestamp.
    except Exception as e:
        print(f"Error posting initial message: {e}")
        return None